In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
from typing import NamedTuple

from census import Census
#from us import states

import pandas as pd

from diabetes_prevention.data.insurance_status import get_insurance_status_prob

In [ ]:
CENSUS_API_KEY = "f286a9da096e259ea2710755630c60bc46fabba0"

Get API key from the [US Census website](https://api.census.gov/data/key_signup.html)

In [ ]:
c = Census(CENSUS_API_KEY)

In [ ]:
nv_dat = get_insurance_status_prob(c, '32')

TODO: Compare to values from PolicyMap

In [ ]:
dat.head(1)

In [ ]:
pm_dat = (
    pd.read_csv(
        'data/insurance_status.csv',
        dtype={
            'census_tract': str,
            'prob_no_insurance': float,
            'prob_medicaid_insurance': float,
            'prob_medicare_insurance': float,
            'prob_private_insurance': float,
        }
    )
    .rename(columns={
        'prob_no_insurance': 'no_insurance_pm',
        'prob_medicaid_insurance': 'medicaid_pm',
        'prob_medicare_insurance': 'medicare_pm',
        'prob_private_insurance': 'private_pm',
    })
)

In [ ]:
dat.dtypes

In [ ]:
(
    pm_dat
    .merge(nv_dat, on='census_tract', how='outer')
    .assign(no_insurance_diff=lambda df: df['no_insurance_pm'] - df['no_insurance'])
    .assign(medicare_diff=lambda df: df['medicare_pm'] - df['medicare'])
    .assign(medicaid_diff=lambda df: df['medicaid_pm'] - df['medicaid'])
    .assign(private_diff=lambda df: df['private_pm'] - df['private'])
    .filter(regex='.*_diff')
    .to_csv('tmp.csv')
)

In [ ]:
[x.name for x in var_groups]

In [ ]:
target_cols = [x.name for x in var_groups if x.name != 'population']
(
    dat
    .assign(tot=lambda df: df[target_cols].sum(1))
    .pipe(lambda df: (
        df.assign(**{
            c: df[c] / df['tot'] 
            for c in target_cols
        })
    ))
    .loc[:, ['census_tract'] + target_cols]
)

In [ ]:
dat['population']

In [ ]:
def  _download_census_tract_var_data(
    var_names: tuple[str], state_fips: str, county_fips: str, delay: bool = True
):
    def delay_time():
        return 2.0 + np.random.uniform(-0.5, 0.5)

    if delay:
        sleep(delay_time())
    return c.acs5.get(
        var_names,
        geo={"for": "tract:*", "in": f"state: {state_fips} county: {county_fips}"},
    )

In [ ]:
state_fips = "50"
census_variable_names = tuple(
    f"{v.name}{'E'}" for v in chain(*(vg.variables for vg in var_groups))
)
dat_df = pd.concat(
    [
        (
            pd.DataFrame.from_records(
                download_census_tract_var_data(census_variable_names, state_fips, county_fips)
            )
            .assign(
                census_tract=lambda df: df["state"]
                .str.cat(df["county"])
                .str.cat(df["tract"])
            )
            .drop(columns=["state", "county", "tract"])
        )
        for county_fips in get_state_county_data(state_fips).county_fips
    ]
)

In [ ]:
raw_var_cols = [x for x in dat_df if x != "census_tract"]
for vg in var_groups:
    dat_df = dat_df.pipe(
        lambda df: df.assign(
            **{vg.name: df[[f"{v.name}{'E'}" for v in vg.variables]].sum(1)}
        )
    )

In [ ]:
tmp_df = dat_df.pipe(lambda df: df.drop(columns=[x for x in df.columns if x not in [vg.name for vg in var_groups]]))

In [ ]:
(
    tmp_df
    .assign(med_sum=lambda df: df.drop(columns='population').sum(1))
    .assign(pct_inc=lambda df: df['med_sum'] / df['population']  * 100)
)